<a href="https://colab.research.google.com/github/Jacquesjh/Absenteeism/blob/main/Absenteesim_Modeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Como sempre, começamos verificando nossa pasta de trabalho

In [65]:
%cd /content/drive/MyDrive/Machine Learning/Absenteeism

/content/drive/MyDrive/Machine Learning/Absenteeism


#Agora podemos importar as bibliotecas que usaremos

In [66]:
import pandas as pd
import numpy as np
import tensorflow as tf

#Podemos importar nossos dados pré processados

In [79]:
npz = np.load('Absenteeism_train_data.npz')

train_inputs = npz['inputs']
train_targets = npz['targets']

In [80]:
npz = np.load('Absenteeism_validation_data.npz')

validation_inputs = npz['inputs']
validation_targets = npz['targets']

In [81]:
npz = np.load('Absenteeism_test_data.npz')

test_inputs = npz['inputs']
test_targets = npz['targets']

In [170]:
train_inputs = np.concatenate((train_inputs, validation_inputs))

In [172]:
train_targets = np.concatenate((train_targets, validation_targets))

#Nosso modelo será um regressor logistico, iremos importá-lo do módulo de Sklearn.

In [166]:
from sklearn.linear_model import LogisticRegressionCV
from sklearn import metrics

In [167]:
reg = LogisticRegressionCV()

In [173]:
reg.fit(train_inputs, train_targets)

LogisticRegressionCV(Cs=10, class_weight=None, cv=None, dual=False,
                     fit_intercept=True, intercept_scaling=1.0, l1_ratios=None,
                     max_iter=100, multi_class='auto', n_jobs=None,
                     penalty='l2', random_state=None, refit=True, scoring=None,
                     solver='lbfgs', tol=0.0001, verbose=0)

In [174]:
reg.score(train_inputs, train_targets)

0.7710651828298887